# 0. Run fastdup and kmeans

In [10]:
import fastdup

In [11]:
# run fastdup
fastdup.run(input_dir = "./Kartaview_Dataset", work_dir = 'Karta_out')

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
Going to loop over dir Kartaview_Dataset
Found total 11084 images to run on
Found total 11084 images to run on■■■■■■■■■■■■■■■■■] 100% Estimated: 0 Minutes 0 Features
3790) Finished write_index() NN model
Stored nn model index file Karta_out/nnf.index
Total time took 344213 ms
Found a total of 165 fully identical images (d>0.990), which are 0.50 %
Found a total of 490 nearly identical images(d>0.980), which are 1.47 %
Found a total of 13327 above threshold images (d>0.900), which are 40.08 %
Found a total of 1108 outlier images         (d<0.050), which are 3.33 %
Min distance found 0.637 max distance 0.997
1675891110 : INFO:     (add_vertices:460): Num vertices for group 0: 11084
1675891110 : INFO:     (commit_edge_buffer:609): In commit edge buffer (0,0)
1675891110 : INFO:     (commit_edge_buffer:680): Shuffling edges ...
1675891110 : INFO:     (commit_edge_buffer:688): Done shuffling edges in 0.017814 secs
1675

0

In [12]:
# run kmeans by fastdup
fastdup.run_kmeans(input_dir = "./Kartaview_Dataset", work_dir = 'Karta_out', 
                   num_clusters = 26, nearest_neighbors_k = 1)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
Going to loop over dir Kartaview_Dataset
Found total 11084 images to run on
Found total 11084 images to run on■■■■■■■■■■■■■■■■■] 100% Estimated: 0 Minutes 0 Features
Wrote total of 26 centroids 
Computed kmeans 26 cluster assignments for 11084 images
Largest cluster has 713 data points, smallest cluster has 0 data points
Average cluster size 426.307678
Total time took 60205 ms
Min distance found 1.000 max distance 278.868


0

# 1. Create clusters based on csv file

## 1.1 Read kmeans_assignments.csv file

In [1]:
import pandas as pd

In [2]:
# read 'kmeans_assignments.csv' and save as 'df'
df = pd.read_csv("~/Documents/New Fast Image Analysis/Karta_out/kmeans_assignments.csv")

In [3]:
print(df.head())

                                            filename  cluster    distance
0  Kartaview_Dataset/Rest_set/2913090_8a751_5f36c...        3  278.868042
1  Kartaview_Dataset/Rest_set/367210_9cb47_592842...        7  269.164673
2  Kartaview_Dataset/Basic_set/3681045_34381a90ec...        6  266.153778
3  Kartaview_Dataset/Rest_set/1372846_750a8_5cb0f...       18  257.131470
4  Kartaview_Dataset/Rest_set/1372846_5ec89_5cb0f...       18  250.680054


In [4]:
# count row
count_row = len(df.index)
print(count_row)

11084


## 1.2 Rank based on cluster

In [6]:
# creating a rank column and passing the returned rank series
df["Rank"] = df["cluster"].rank()

In [7]:
print(df.head())

                                            filename  cluster    distance  \
0  Kartaview_Dataset/Rest_set/2913090_8a751_5f36c...        3  278.868042   
1  Kartaview_Dataset/Rest_set/367210_9cb47_592842...        7  269.164673   
2  Kartaview_Dataset/Basic_set/3681045_34381a90ec...        6  266.153778   
3  Kartaview_Dataset/Rest_set/1372846_750a8_5cb0f...       18  257.131470   
4  Kartaview_Dataset/Rest_set/1372846_5ec89_5cb0f...       18  250.680054   

     Rank  
0  1138.5  
1  3096.0  
2  2555.5  
3  8244.5  
4  8244.5  


In [8]:
# sorting w.r.t Rank column
df.sort_values("Rank", inplace = True)

In [9]:
print(df.head())

                                                filename  cluster    distance  \
1398   Kartaview_Dataset/Rest_set/3346325_38b21_5ff10...        0  112.855469   
2074   Kartaview_Dataset/Rest_set/3346325_26fbe_5ff10...        0  103.636719   
10230  Kartaview_Dataset/Rest_set/3640973_74d41_60bc3...        0   47.200378   
10650  Kartaview_Dataset/Rest_set/3640973_4af06_60bc3...        0   40.126465   
4436   Kartaview_Dataset/Rest_set/2918366_4d98b_5f383...        0   85.230164   

       Rank  
1398   73.5  
2074   73.5  
10230  73.5  
10650  73.5  
4436   73.5  


In [21]:
# export 'df' as 'kmeans_after_rank.csv'
df.to_csv('~/Documents/New Fast Image Analysis/kmeans_after_rank.csv')

## 1.3 Split df into multiple dataframes

In [10]:
# Drop Columns 'distance' and 'Rank'
df.drop(['distance', 'Rank'], axis=1, inplace = True)

In [11]:
# create unique list of clusters
UniqueClusters = df.cluster.unique()

In [12]:
# create a data frame dictionary to store your data frames
DataFrameDict = {elem : pd.DataFrame() for elem in UniqueClusters}

for key in DataFrameDict.keys():
    DataFrameDict[key] = df[:][df.cluster == key]

In [13]:
print(DataFrameDict[0].values[0])
print(DataFrameDict[0].values[0][0])
print(DataFrameDict[0].values[1][0])
print(len(DataFrameDict[0]))
print(type(DataFrameDict[0].values))

['Kartaview_Dataset/Rest_set/3346325_38b21_5ff1052f75a8a.jpg' 0]
Kartaview_Dataset/Rest_set/3346325_38b21_5ff1052f75a8a.jpg
Kartaview_Dataset/Rest_set/3346325_26fbe_5ff10539426fc.jpg
146
<class 'numpy.ndarray'>


## 1.4 Delete cluster id and fulfill image address

In [14]:
import numpy as np

In [15]:
# create a new data frame dictionary
DataFrameDictNew = {elem : pd.DataFrame() for elem in UniqueClusters}

In [16]:
for key in DataFrameDict.keys():
    # delete 1st column(cluster id)
    DataFrameDictNew[key] = np.delete(DataFrameDict[key].values, 1, 1)
    # fulfill image address
    for i in range(len(DataFrameDictNew[key])):
        DataFrameDictNew[key][i] = '~/Documents/New Fast Image Analysis/' + DataFrameDictNew[key][i]

In [24]:
print(type(DataFrameDictNew))

<class 'dict'>


In [17]:
print(type(DataFrameDictNew[0]))

<class 'numpy.ndarray'>


In [19]:
print(DataFrameDictNew[0])

[['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3346325_38b21_5ff1052f75a8a.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3346325_26fbe_5ff10539426fc.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3640973_74d41_60bc39a527e14.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3640973_4af06_60bc39b89a7c8.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/2918366_4d98b_5f38359d8f113.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/2913090_8d951_5f36c96d04621.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3640973_a1267_60bc39c64609c.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/2819646_36125_5f1dc15b17173.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3493677_fad83_607162f5c20c6.jpg']
 ['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3235117_60842_5fabf55a70773.jpg']


## 1.5 Convert Array to list and flatten it

In [21]:
# flatten function
def flatten(l):
    return [item for sublist in l for item in sublist]

In [25]:
# convert Array to list and flatten
for key in DataFrameDictNew.keys():
    DataFrameDictNew[key] = flatten(DataFrameDictNew[key].tolist())

In [26]:
print(DataFrameDictNew[0])

['~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3346325_38b21_5ff1052f75a8a.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3346325_26fbe_5ff10539426fc.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3640973_74d41_60bc39a527e14.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3640973_4af06_60bc39b89a7c8.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/2918366_4d98b_5f38359d8f113.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/2913090_8d951_5f36c96d04621.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3640973_a1267_60bc39c64609c.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/2819646_36125_5f1dc15b17173.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3493677_fad83_607162f5c20c6.jpg', '~/Documents/New Fast Image Analysis/Kartaview_Dataset/Rest_set/3235117_60842_5fabf55a70773.jpg', '~/Documents/New Fa

## 1.5 Load Image into Fiftyone and compute Uniqueness

In [28]:
import fiftyone as fo
import fiftyone.brain as fob

In [54]:
# uniqueness function
def compute_uniqueness(image_address):
    # load image into fiftyone based on DataFrameDictNew value
    dataset = fo.Dataset.from_images(image_address)
    # compute uniqueness
    fob.compute_uniqueness(dataset)
    # save filepath and uniqueness score as array 'uniq'
    uniq = np.empty((0,2))
    for sample in dataset:
        temp = np.array([[sample["filepath"], sample["uniqueness"]]])
        uniq = np.append(uniq, temp, axis=0)
    # convert array 'unique' to dataframe 'df_temp'
    df_temp = pd.DataFrame(uniq, columns=['filepath', 'uniqueness'])
    # rank based on uniqueness score
    df_temp.sort_values("uniqueness", ascending = False, inplace = True)
    return df_temp

In [55]:
# create a new data frame dictionary
DataFrameDictUni = {elem : pd.DataFrame() for elem in UniqueClusters}

In [56]:
# call uniqueness function
for key in DataFrameDict.keys():
    DataFrameDictUni[key] = compute_uniqueness(DataFrameDictNew[key])

 100% |█████████████████| 146/146 [42.3ms elapsed, 0s remaining, 3.5K samples/s]   
Computing embeddings...
 100% |█████████████████| 146/146 [843.7ms elapsed, 0s remaining, 173.0 samples/s]      
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 435/435 [89.7ms elapsed, 0s remaining, 4.9K samples/s]    
Computing embeddings...
  26% |████/------------| 112/435 [9.2s elapsed, 26.5s remaining, 12.2 samples/s] 

/home/yiwei/anaconda3/lib/python3.9/site-packages/fiftyone/utils/torch.py:689: RuntimeWarning: overflow encountered in exp
  odds = np.exp(logits)
/home/yiwei/anaconda3/lib/python3.9/site-packages/fiftyone/utils/torch.py:690: RuntimeWarning: invalid value encountered in true_divide
  odds /= np.sum(odds, axis=1, keepdims=True)


 100% |█████████████████| 435/435 [19.6s elapsed, 0s remaining, 22.2 samples/s]    
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 338/338 [75.1ms elapsed, 0s remaining, 4.5K samples/s]   
Computing embeddings...
 100% |█████████████████| 338/338 [11.7s elapsed, 0s remaining, 28.8 samples/s]   
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 438/438 [60.5ms elapsed, 0s remaining, 7.2K samples/s]   
Computing embeddings...
  62% |██████████|------| 272/438 [11.4s elapsed, 6.9s remaining, 23.9 samples/s]  

/home/yiwei/anaconda3/lib/python3.9/site-packages/fiftyone/utils/torch.py:689: RuntimeWarning: overflow encountered in exp
  odds = np.exp(logits)
/home/yiwei/anaconda3/lib/python3.9/site-packages/fiftyone/utils/torch.py:690: RuntimeWarning: invalid value encountered in true_divide
  odds /= np.sum(odds, axis=1, keepdims=True)


 100% |█████████████████| 438/438 [16.6s elapsed, 0s remaining, 34.5 samples/s]      
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 670/670 [83.3ms elapsed, 0s remaining, 8.0K samples/s]    
Computing embeddings...
 100% |█████████████████| 670/670 [24.2s elapsed, 0s remaining, 35.9 samples/s]    
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 194/194 [64.6ms elapsed, 0s remaining, 3.0K samples/s]   
Computing embeddings...
 100% |█████████████████| 194/194 [8.2s elapsed, 0s remaining, 23.6 samples/s]   
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 668/668 [108.3ms elapsed, 0s remaining, 6.2K samples/s]   
Computing embeddings...
 100% |█████████████████| 668/668 [22.1s elapsed, 0s remaining, 34.9 samples/s]    
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 413/413 [60.9ms elapsed, 0s remaining, 6.8K samples/s]   
Computing embeddings...
 100% 

...

 100% |█████████████████| 337/337 [13.8s elapsed, 0s remaining, 24.4 samples/s]   
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 139/139 [38.8ms elapsed, 0s remaining, 3.6K samples/s]   
Computing embeddings...
 100% |█████████████████| 139/139 [7.2s elapsed, 0s remaining, 19.4 samples/s]   
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 667/667 [118.4ms elapsed, 0s remaining, 5.6K samples/s]   
Computing embeddings...
 100% |█████████████████| 667/667 [25.4s elapsed, 0s remaining, 33.9 samples/s]      
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 466/466 [87.7ms elapsed, 0s remaining, 5.3K samples/s]    
Computing embeddings...
  69% |███████████------| 320/466 [14.5s elapsed, 6.2s remaining, 24.3 samples/s]  

/home/yiwei/anaconda3/lib/python3.9/site-packages/fiftyone/utils/torch.py:689: RuntimeWarning: overflow encountered in exp
  odds = np.exp(logits)
/home/yiwei/anaconda3/lib/python3.9/site-packages/fiftyone/utils/torch.py:690: RuntimeWarning: invalid value encountered in true_divide
  odds /= np.sum(odds, axis=1, keepdims=True)


 100% |█████████████████| 466/466 [18.2s elapsed, 0s remaining, 29.3 samples/s]    
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 382/382 [79.1ms elapsed, 0s remaining, 4.8K samples/s]    
Computing embeddings...
  63% |██████████-------| 240/382 [15.5s elapsed, 9.1s remaining, 15.5 samples/s]  

/home/yiwei/anaconda3/lib/python3.9/site-packages/fiftyone/utils/torch.py:689: RuntimeWarning: overflow encountered in exp
  odds = np.exp(logits)
/home/yiwei/anaconda3/lib/python3.9/site-packages/fiftyone/utils/torch.py:690: RuntimeWarning: invalid value encountered in true_divide
  odds /= np.sum(odds, axis=1, keepdims=True)


 100% |█████████████████| 382/382 [19.6s elapsed, 0s remaining, 22.9 samples/s]    
Computing uniqueness...
Uniqueness computation complete
 100% |█████████████████| 182/182 [34.9ms elapsed, 0s remaining, 5.2K samples/s]   
Computing embeddings...
 100% |█████████████████| 182/182 [6.8s elapsed, 0s remaining, 26.6 samples/s]      
Computing uniqueness...
Uniqueness computation complete


In [57]:
print(type(DataFrameDictUni[0]))

<class 'pandas.core.frame.DataFrame'>


In [58]:
print(DataFrameDictUni[0])

                                              filepath           uniqueness
122  /home/yiwei/Documents/New Fast Image Analysis/...                  1.0
135  /home/yiwei/Documents/New Fast Image Analysis/...   0.9688198260532347
73   /home/yiwei/Documents/New Fast Image Analysis/...   0.9642063995077501
88   /home/yiwei/Documents/New Fast Image Analysis/...   0.9095245072178738
30   /home/yiwei/Documents/New Fast Image Analysis/...   0.8322548392150948
..                                                 ...                  ...
69   /home/yiwei/Documents/New Fast Image Analysis/...  0.07695671328315233
107  /home/yiwei/Documents/New Fast Image Analysis/...   0.0762952370280206
65   /home/yiwei/Documents/New Fast Image Analysis/...   0.0723905492530136
75   /home/yiwei/Documents/New Fast Image Analysis/...  0.07095399559412453
101  /home/yiwei/Documents/New Fast Image Analysis/...  0.06954876837096353

[146 rows x 2 columns]


## 1.6 Export Uniqueness to csv file

In [59]:
import os

In [60]:
# make parent folder
uniqueness_dir = '/home/yiwei/Documents/New Fast Image Analysis/Karta_uniqueness_out'
os.makedirs(uniqueness_dir, exist_ok=True)

In [61]:
# create '0~25.csv' file based on clusters
for key in DataFrameDictUni.keys():
    file_path = uniqueness_dir + '/' + str(key) + '.csv'
    DataFrameDictUni[key].to_csv(file_path)

In [93]:
# Add all filepath into dataframe 'df_all_uniqueness'
df_all_uniqueness = pd.DataFrame()
for key in DataFrameDictUni.keys():
    df_all_uniqueness = pd.concat([df_all_uniqueness, pd.DataFrame(DataFrameDictUni[key]['filepath'].values, columns=[str(key)])], axis=1)
print(df_all_uniqueness)

                                                     0  \
0    /home/yiwei/Documents/New Fast Image Analysis/...   
1    /home/yiwei/Documents/New Fast Image Analysis/...   
2    /home/yiwei/Documents/New Fast Image Analysis/...   
3    /home/yiwei/Documents/New Fast Image Analysis/...   
4    /home/yiwei/Documents/New Fast Image Analysis/...   
..                                                 ...   
708                                                NaN   
709                                                NaN   
710                                                NaN   
711                                                NaN   
712                                                NaN   

                                                     1  \
0    /home/yiwei/Documents/New Fast Image Analysis/...   
1    /home/yiwei/Documents/New Fast Image Analysis/...   
2    /home/yiwei/Documents/New Fast Image Analysis/...   
3    /home/yiwei/Documents/New Fast Image Analysis/...   
4    /home/yi

In [94]:
# save dataframe 'df_all_uniqueness' as 'all.csv'
df_all_uniqueness.to_csv(uniqueness_dir + '/all.csv')